This Notebook is to try to fill in missing values, where possible. 

In [1]:
import requests
import json
import csv
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import random
import os
import string
from time import sleep
from dotenv import load_dotenv
from tqdm import tqdm, tqdm_notebook
from ratelimit import limits, sleep_and_retry
load_dotenv();

In [2]:
tqdm.pandas()

In [3]:
books = pd.read_csv('book_data_clean.csv')

First, let's drop our duplicate rows.

In [4]:
len(books)

24971

In [5]:
books = books.drop_duplicates()

In [6]:
len(books)

24861

In [7]:
books.head()

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year
0,The Vanished Birds,Simon Jimenez,124205.0,55.18,6.37,1.95,0.36,1.58,"['Science Fiction', 'Fiction', 'Fantasy', 'Que...",2020.0
1,The Price of Honor,Jonathan P. Brazee,77253.0,35.35,8.71,2.63,0.71,1.92,['Science Fiction'],2017.0
2,The Mathematical Murder of Innocence,Michael Carter,37688.0,24.08,8.11,4.13,1.56,2.58,[],2020.0
3,The Case of the Baker Street Irregulars,Anthony Boucher,80557.0,32.33,8.41,3.72,1.64,2.08,"['Mystery', 'Fiction', 'Crime', 'Humor', 'Clas...",1940.0
4,Zombie Nation,Charlie Dalton,64396.0,51.11,8.22,2.21,0.58,1.63,[],2020.0


I've noticed that some books are missing Prosecraft data. Let's find why.

In [8]:
missing_prosecraft = books[books['vividness'].isna()].copy()

In [9]:
len(missing_prosecraft)

410

In [10]:
missing_prosecraft.head()

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year
176,Infinite Baseball,Alva No',NaN,NaN,NaN,NaN,NaN,NaN,"['Baseball', 'Sports', 'Nonfiction', 'Philosop...",2019.0
233,The Little Buddhist Monk,César Aira,NaN,NaN,NaN,NaN,NaN,NaN,"['Fiction', 'Latin American', 'Contemporary', ...",2017.0
329,The Fire Engine That Disappeared,Maj Sjöwall & Per Wahlöö,NaN,NaN,NaN,NaN,NaN,NaN,"['Mystery', 'Crime', 'Fiction', 'Scandinavian ...",1969.0
388,How to Turn Into a Bird,María José Ferrada,NaN,NaN,NaN,NaN,NaN,NaN,"['Fiction', 'Contemporary', 'Coming Of Age', '...",2022.0
588,The Silence of the White City,Eva García Sáenz,NaN,NaN,NaN,NaN,NaN,NaN,"['Thriller', 'Mystery', 'Crime', 'Fiction', 'S...",2016.0


What these seem to have in common is that they contain special characters. When I check the URL of the books on Prosecraft, the accents do not appear. That's likely where the error occurred. First, I find all the characters that appear in titles

In [11]:
alphanum = set('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890 ')

In [12]:
print(set(missing_prosecraft.title.sum()+missing_prosecraft.author.sum()).difference(alphanum))

{'&', 'ï', '̈', '+', 'ä', '/', '?', '̧', ')', ',', 'ô', 'ë', '̂', 'ó', 'ō', '̃', 'Ø', 'é', '‘', 'Æ', '̊', '̀', '*', '[', 'ö', "'", 'à', 'í', ';', ':', 'ł', '%', 'â', 'ñ', '!', ']', 'ç', 'ø', 'ž', '̇', '̌', '’', '(', '.', '$', 'á', '°', 'ð', 'Ž', 'è', 'ü', '́', '#', '-', 'ū', '–'}


In [13]:
#Checked how each of the accented letter characters maps into a Prosecraft URL
replacements = {'à':'a','é':'e','â':'a','ç':'c','ñ':'n','Ž':'Z',
                'ž':'z','è':'e','ö':'o','á':'a','ó':'o','ū':'u',
                'í':'i','ô':'o','Ø':'O','ł':'l','ä':'a','ï':'i',
                'ë':'e','ü':'u','ō':'o','ð':'d','ć':'c','É':'E',
                'Ż':'Z','ô':'o','ş':'s','ũ':'u','Š':'S','ř':'r',
                'č':'c','ĩ':'i', 'š':'s','Æ':'Ae'}

In [14]:
missing_prosecraft['author_clean'] = missing_prosecraft['author']

In [15]:
for before, after in replacements.items():
    missing_prosecraft.author_clean = missing_prosecraft.author_clean.str.replace(before,after)

In [16]:
pd.concat([missing_prosecraft[['author','author_clean']].head(),(missing_prosecraft[['author','author_clean']].tail())])

,author,author_clean
176,Alva No',Alva No'
233,César Aira,César Aira
329,Maj Sjöwall & Per Wahlöö,Maj Sjowall & Per Wahloo
388,María José Ferrada,María José Ferrada
588,Eva García Sáenz,Eva García Sáenz
24739,Ragnar Jónasson,Ragnar Jónasson
24745,María Amparo Ruiz de Burton,María Amparo Ruiz de Burton
24783,M. L. Longworth,M. L. Longworth
24955,Tor Fleck,Tor Fleck
24970,Joe Schrieber,Joe Schrieber


Hmm... not all of the accents are gone. Let's investigate why!

In [17]:
print(set(missing_prosecraft.author_clean.sum()).difference(alphanum))

{'&', '̈', "'", 'ø', '(', '.', ')', ',', '̊', '́', '̃', '-', '̀', '’'}


In [18]:
missing_prosecraft.iloc[1].author.replace('é','e')

'César Aira'

Strange! It's almost as if it's a different character entirely. So I tried copy/pasting the accented e directly from the previous line's output, and...

In [19]:
missing_prosecraft.iloc[1].author.replace('é','e')

'Cesar Aira'

Well, here's your problem!

In [20]:
'é' == 'é'

False

Let's try running this again with the accented letters that haven't changed replaced with the correct versions, copied directly from the DataFrame

In [21]:
corrected_replacements = {'á':'a','é':'e','ó':'o','ë':'e','í':'i','ü':'u',
                          'ñ':'n','ú':'u','Á':'A','Ó':'O','è':'e','ö':'o',
                          'ä':'a','Ö':'O','ø':'o','ï':'i','ò':'o','Ü':'U',
                          'à':'a','å':'a','Å':'A'}

In [22]:
for before, after in corrected_replacements.items():
    missing_prosecraft.author_clean = missing_prosecraft.author_clean.str.replace(before,after)

In [23]:
pd.concat([missing_prosecraft[['author','author_clean']].head(),(missing_prosecraft[['author','author_clean']].tail())])

,author,author_clean
176,Alva No',Alva No'
233,César Aira,Cesar Aira
329,Maj Sjöwall & Per Wahlöö,Maj Sjowall & Per Wahloo
388,María José Ferrada,Maria Jose Ferrada
588,Eva García Sáenz,Eva Garcia Saenz
24739,Ragnar Jónasson,Ragnar Jonasson
24745,María Amparo Ruiz de Burton,Maria Amparo Ruiz de Burton
24783,M. L. Longworth,M. L. Longworth
24955,Tor Fleck,Tor Fleck
24970,Joe Schrieber,Joe Schrieber


Now let's run it for the titles!

In [24]:
missing_prosecraft['title_clean'] = missing_prosecraft['title']

In [25]:
replacements.update(corrected_replacements)

In [26]:
for before, after in (replacements).items():
    missing_prosecraft.title_clean = missing_prosecraft.title_clean.str.replace(before,after)

In [27]:
#A few replacements made specifically in Prosecraft URLs
more_replacements = {'*':'-','&':'and'}

In [28]:
for before, after in (more_replacements).items():
    missing_prosecraft.title_clean = missing_prosecraft.title_clean.str.replace(before,after)
    missing_prosecraft.author_clean = missing_prosecraft.author_clean.str.replace(before,after)

In [29]:
pd.concat([missing_prosecraft[['title','title_clean']].head(),(missing_prosecraft[['title','title_clean']].tail())])

,title,title_clean
176,Infinite Baseball,Infinite Baseball
233,The Little Buddhist Monk,The Little Buddhist Monk
329,The Fire Engine That Disappeared,The Fire Engine That Disappeared
388,How to Turn Into a Bird,How to Turn Into a Bird
588,The Silence of the White City,The Silence of the White City
24739,The Mist,The Mist
24745,The Squatter and the Don,The Squatter and the Don
24783,A Noël Killing,A Noel Killing
24955,Agency ‘O’,Agency ‘O’
24970,Star Wars - The Mandalorian: Junior Novel,Star Wars - The Mandalorian: Junior Novel


We still have a number of special characters left that are probably giving us trouble. 

In [30]:
specials = set(missing_prosecraft.title_clean.sum()+missing_prosecraft.author_clean.sum()).difference(alphanum)
print(specials)

{'+', '/', '?', ')', ',', '‘', '[', "'", ';', ':', '%', '!', ']', '’', '(', '.', '$', '°', '#', '-', '–'}


In [31]:
special_df = pd.DataFrame()

In [32]:
#Creates a dataframe with only titles/authors whose cleaned version has special characters
for special in specials:
    special_df = pd.concat([special_df,missing_prosecraft[(missing_prosecraft['title_clean'].str.contains('\\'+special)) | 
                             (missing_prosecraft['author_clean'].str.contains('\\'+special))]])

In [33]:
print(len(special_df))
pd.concat([special_df.head(),special_df.tail()])


398


,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year,author_clean,title_clean
3402,Queer Intentions: A (Personal) journey through...,Amelia Abraham,NaN,NaN,NaN,NaN,NaN,NaN,"['Nonfiction', 'LGBT', 'Queer', 'Memoir', 'Aud...",2019.0,Amelia Abraham,Queer Intentions: A (Personal) journey through...
8834,X + Y,Eugenia Cheng,NaN,NaN,NaN,NaN,NaN,NaN,"['Nonfiction', 'Feminism', 'Science', 'Gender'...",2020.0,Eugenia Cheng,X + Y
17712,1000+ Little Habits of Happy Successful Relati...,Marc Chernoff & Angel Chernoff,NaN,NaN,NaN,NaN,NaN,NaN,"['Nonfiction', 'Personal Development', 'Self H...",2021.0,Marc Chernoff and Angel Chernoff,1000+ Little Habits of Happy Successful Relati...
20841,I Travel By Night / Last Train From Perdition,Robert McCammon,NaN,NaN,NaN,NaN,NaN,NaN,['Horror'],2022.0,Robert McCammon,I Travel By Night / Last Train From Perdition
2324,Why? Explaining the Holocaust,Peter Hayes,NaN,NaN,NaN,NaN,NaN,NaN,"['History', 'Nonfiction', 'Holocaust', 'World ...",2017.0,Peter Hayes,Why? Explaining the Holocaust
23046,How To Have Kick-Ass Ideas,Chris Baréz-Brown,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,Chris Barez-Brown,How To Have Kick-Ass Ideas
23777,Star Trek - Voyager: To Lose the Earth,Kirsten Beyer,NaN,NaN,NaN,NaN,NaN,NaN,"['Star Trek', 'Science Fiction', 'Star Trek Vo...",2019.0,Kirsten Beyer,Star Trek - Voyager: To Lose the Earth
24666,Star Wars - Alphabet Squadron: Victory’s Price,Alexander Freed,NaN,NaN,NaN,NaN,NaN,NaN,"['Star Wars', 'Science Fiction', 'Fiction', 'F...",2021.0,Alexander Freed,Star Wars - Alphabet Squadron: Victory’s Price
24970,Star Wars - The Mandalorian: Junior Novel,Joe Schrieber,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,Joe Schrieber,Star Wars - The Mandalorian: Junior Novel
7890,The Mind–Body Problem,Jonathan Westphal,NaN,NaN,NaN,NaN,NaN,NaN,"['Philosophy', 'Nonfiction', 'Science', 'Psych...",2016.0,Jonathan Westphal,The Mind–Body Problem


In [34]:
for special in specials:
    missing_prosecraft.title_clean = missing_prosecraft.title_clean.str.replace(special,'')
    missing_prosecraft.author_clean = missing_prosecraft.author_clean.str.replace(special,'')

In [35]:
missing_prosecraft.head()

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year,author_clean,title_clean
176,Infinite Baseball,Alva No',NaN,NaN,NaN,NaN,NaN,NaN,"['Baseball', 'Sports', 'Nonfiction', 'Philosop...",2019.0,Alva No,Infinite Baseball
233,The Little Buddhist Monk,César Aira,NaN,NaN,NaN,NaN,NaN,NaN,"['Fiction', 'Latin American', 'Contemporary', ...",2017.0,Cesar Aira,The Little Buddhist Monk
329,The Fire Engine That Disappeared,Maj Sjöwall & Per Wahlöö,NaN,NaN,NaN,NaN,NaN,NaN,"['Mystery', 'Crime', 'Fiction', 'Scandinavian ...",1969.0,Maj Sjowall and Per Wahloo,The Fire Engine That Disappeared
388,How to Turn Into a Bird,María José Ferrada,NaN,NaN,NaN,NaN,NaN,NaN,"['Fiction', 'Contemporary', 'Coming Of Age', '...",2022.0,Maria Jose Ferrada,How to Turn Into a Bird
588,The Silence of the White City,Eva García Sáenz,NaN,NaN,NaN,NaN,NaN,NaN,"['Thriller', 'Mystery', 'Crime', 'Fiction', 'S...",2016.0,Eva Garcia Saenz,The Silence of the White City


In [36]:
#Find any non-alphanumerical special characters I've missed
specials = set(missing_prosecraft.title_clean.sum()+missing_prosecraft.author_clean.sum()).difference(alphanum)
print(specials)

set()


In [37]:
pd.concat([missing_prosecraft[['title_clean','author_clean']].head(),(missing_prosecraft[['title_clean','author_clean']].tail())])

,title_clean,author_clean
176,Infinite Baseball,Alva No
233,The Little Buddhist Monk,Cesar Aira
329,The Fire Engine That Disappeared,Maj Sjowall and Per Wahloo
388,How to Turn Into a Bird,Maria Jose Ferrada
588,The Silence of the White City,Eva Garcia Saenz
24739,The Mist,Ragnar Jonasson
24745,The Squatter and the Don,Maria Amparo Ruiz de Burton
24783,A Noel Killing,M L Longworth
24955,Agency O,Tor Fleck
24970,Star Wars The Mandalorian Junior Novel,Joe Schrieber


In [38]:
#recreate DataFrame of titles/authors involving special characters. 
#should be empty if I've done my job!
special_df = pd.DataFrame()

In [39]:
for special in specials:
    special_df = pd.concat([special_df,missing_prosecraft[(missing_prosecraft['title_clean'].str.contains('\\'+special)) | 
                             (missing_prosecraft['author_clean'].str.contains('\\'+special))]])

In [40]:
print(len(special_df))
special_df.head()

0


""


Now we're ready to scrape the data from these 410 books! 

In [41]:
@sleep_and_retry
@limits(calls=9, period=60)
def get_prosecraft(index):
    '''Given the cleaned title and author of a book, return the Prosecraft linguistic data'''
    
    sleep(random.uniform(0.5,1.5))
    
    #Copies the non-prosecraft columns of the original dataframe
    #row = missing_prosecraft[(missing_prosecraft['title_clean'] == title) & (missing_prosecraft['author_clean'] == author)]
    row = missing_prosecraft.loc[index]
    info = {'title': row['title'], 'author':row['author'], 
            'genre':row['genre'],'year':row['year']}
    
    #Formats the URL
    title = row['title_clean'].replace(' ','-').lower()
    author = row['author_clean'].replace(' ','-').lower()
    URL = f"http://prosecraft.io/library/{author}/{title}"
    
    
    #Fills out the missing elements of the original dataframe
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    headings = soup.find_all("div", {"class": "book-info-metric-heading"})
    values = soup.find_all("div", {"class": "book-info-metric-value"})
    for heading, value in zip(headings, values):
        info[heading.text] = float(value.text.strip('%').replace(',',''))
        
    
    return pd.Series(info)
    

In [42]:
new_df = missing_prosecraft.progress_apply(lambda row: get_prosecraft(row.name), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████| 410/410 [45:07<00:00,  6.60s/it]


In [43]:
#Make sure new dataframe has its columns in the right order
in_order = new_df[['title', 'author', 'total words', 'vividness', 'passive voice',
       'all adverbs', 'ly-adverbs', 'non-ly-adverbs', 'genre', 'year']]

In [44]:
#Let's put the new data onto the old, with the ones still missing prosecraft data dropped
merged = pd.concat([books,in_order]).dropna(subset=['vividness'])

A few cells to double check that the merge was successful.

In [45]:
#Length of original dataframe
len(books)

24861

In [46]:
#Missing prosecraft data in original dataframe
sum(books['vividness'].isna())

410

In [47]:
#Amount of missing prosecraft data we have not fixed
sum(in_order['vividness'].isna())

85

In [48]:
#Length of the new dataframe
len(merged)

24776

In [49]:
#Number of missing prosecraft values in the new dataframe
sum(merged['vividness'].isna())

0

In [50]:
#Ideally, the merged dataframe should be shorter than the books dataframe
#by exactly as many books as we couldn't fix--in other words, 85. 
len(books) - len(merged)

85

In [51]:
#Since all the fixed data is added onto the end, we should see some of it here.
merged.tail()

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year
24667,Other Terrors,"Various Authors (ed, Vince A. Liaguno & Rena M...",102917.0,65.86,6.74,2.36,0.67,1.68,"['Horror', 'Short Stories', 'Anthologies', 'Fi...",2022.0
24739,The Mist,Ragnar Jónasson,63861.0,36.15,10.30,3.79,1.38,2.41,"['Horror', 'Fiction', 'Thriller', 'Fantasy', '...",1980.0
24745,The Squatter and the Don,María Amparo Ruiz de Burton,136680.0,30.19,8.71,3.83,1.09,2.74,"['Fiction', 'Classics', 'School', '19th Centur...",1885.0
24783,A Noël Killing,M. L. Longworth,71960.0,52.44,7.31,2.75,0.90,1.86,[],NaN
24955,Agency ‘O’,Tor Fleck,76068.0,55.25,6.79,2.65,1.05,1.61,[],NaN


In [52]:
#As this csv was made successfully, I'm commenting it out when I rerun. 
#merged.to_csv('no_null_prosecraft.csv')

In [53]:
in_order[in_order['vividness'].isna()].head()

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year
176,Infinite Baseball,Alva No',NaN,NaN,NaN,NaN,NaN,NaN,"['Baseball', 'Sports', 'Nonfiction', 'Philosop...",2019.0
892,ADHD 2.0,Edward M. Hallowell & John J. Ratey,NaN,NaN,NaN,NaN,NaN,NaN,"['Adhd', 'Nonfiction', 'Psychology', 'Self Hel...",2021.0
1491,Walk Through Walls,Marina Abramović,NaN,NaN,NaN,NaN,NaN,NaN,"['Art', 'Nonfiction', 'Memoir', 'Biography', '...",2016.0
1585,D.O.D.O.,Neal Stephenson & Nicole Galland,NaN,NaN,NaN,NaN,NaN,NaN,"['Science Fiction', 'Fantasy', 'Fiction', 'Tim...",2017.0
1729,Zipped,Laura McNeal & Tom McNeal,NaN,NaN,NaN,NaN,NaN,NaN,"['Young Adult', 'Fiction', 'Realistic Fiction'...",2003.0
...,...,...,...,...,...,...,...,...,...,...
23987,The Thirteenth Tale,Diane Setterfield,NaN,NaN,NaN,NaN,NaN,NaN,"['Fiction', 'Mystery', 'Historical Fiction', '...",2006.0
24231,13.8,John Gribbin,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN
24386,(R)Evolution,M. E. Purfield,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN
24666,Star Wars - Alphabet Squadron: Victory’s Price,Alexander Freed,NaN,NaN,NaN,NaN,NaN,NaN,"['Star Wars', 'Science Fiction', 'Fiction', 'F...",2021.0


In [54]:
missing_prosecraft = missing_prosecraft[in_order['vividness'].isna()]

In [55]:
missing_prosecraft.head()

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year,author_clean,title_clean
176,Infinite Baseball,Alva No',NaN,NaN,NaN,NaN,NaN,NaN,"['Baseball', 'Sports', 'Nonfiction', 'Philosop...",2019.0,Alva No,Infinite Baseball
892,ADHD 2.0,Edward M. Hallowell & John J. Ratey,NaN,NaN,NaN,NaN,NaN,NaN,"['Adhd', 'Nonfiction', 'Psychology', 'Self Hel...",2021.0,Edward M Hallowell and John J Ratey,ADHD 20
1491,Walk Through Walls,Marina Abramović,NaN,NaN,NaN,NaN,NaN,NaN,"['Art', 'Nonfiction', 'Memoir', 'Biography', '...",2016.0,Marina Abramovic,Walk Through Walls
1585,D.O.D.O.,Neal Stephenson & Nicole Galland,NaN,NaN,NaN,NaN,NaN,NaN,"['Science Fiction', 'Fantasy', 'Fiction', 'Tim...",2017.0,Neal Stephenson and Nicole Galland,DODO
1729,Zipped,Laura McNeal & Tom McNeal,NaN,NaN,NaN,NaN,NaN,NaN,"['Young Adult', 'Fiction', 'Realistic Fiction'...",2003.0,Laura McNeal and Tom McNeal,Zipped


In [56]:
#This one's a formatting error I caused myself. May as well fix it. 
missing_prosecraft.loc[176, 'author'] = 'Alva Noë'
missing_prosecraft.loc[176, 'author_clean'] = 'Alva Noe'

In [57]:
missing_prosecraft.head()

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year,author_clean,title_clean
176,Infinite Baseball,Alva Noë,NaN,NaN,NaN,NaN,NaN,NaN,"['Baseball', 'Sports', 'Nonfiction', 'Philosop...",2019.0,Alva Noe,Infinite Baseball
892,ADHD 2.0,Edward M. Hallowell & John J. Ratey,NaN,NaN,NaN,NaN,NaN,NaN,"['Adhd', 'Nonfiction', 'Psychology', 'Self Hel...",2021.0,Edward M Hallowell and John J Ratey,ADHD 20
1491,Walk Through Walls,Marina Abramović,NaN,NaN,NaN,NaN,NaN,NaN,"['Art', 'Nonfiction', 'Memoir', 'Biography', '...",2016.0,Marina Abramovic,Walk Through Walls
1585,D.O.D.O.,Neal Stephenson & Nicole Galland,NaN,NaN,NaN,NaN,NaN,NaN,"['Science Fiction', 'Fantasy', 'Fiction', 'Tim...",2017.0,Neal Stephenson and Nicole Galland,DODO
1729,Zipped,Laura McNeal & Tom McNeal,NaN,NaN,NaN,NaN,NaN,NaN,"['Young Adult', 'Fiction', 'Realistic Fiction'...",2003.0,Laura McNeal and Tom McNeal,Zipped


I checked some values and noticed that in titles like ADHD 2.0 and D.O.D.O, the period gets turned into a space, *not* dropped. Let's try fixing that. 

In [58]:
missing_prosecraft[missing_prosecraft['title'].str.contains('\.')].head()

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year,author_clean,title_clean
892,ADHD 2.0,Edward M. Hallowell & John J. Ratey,NaN,NaN,NaN,NaN,NaN,NaN,"['Adhd', 'Nonfiction', 'Psychology', 'Self Hel...",2021.0,Edward M Hallowell and John J Ratey,ADHD 20
1585,D.O.D.O.,Neal Stephenson & Nicole Galland,NaN,NaN,NaN,NaN,NaN,NaN,"['Science Fiction', 'Fantasy', 'Fiction', 'Tim...",2017.0,Neal Stephenson and Nicole Galland,DODO
2578,Corruption in the O.R.,Barbara Ebel,NaN,NaN,NaN,NaN,NaN,NaN,[],2019.0,Barbara Ebel,Corruption in the OR
3390,Life 3.0,Max Tegmark,NaN,NaN,NaN,NaN,NaN,NaN,"['Science', 'Nonfiction', 'Artificial Intellig...",2017.0,Max Tegmark,Life 30
4830,Tough Luck L.A,Murray Sinclair,NaN,NaN,NaN,NaN,NaN,NaN,['Crime'],1981.0,Murray Sinclair,Tough Luck LA
7806,Z.E.D.S. Rising,Bradley Botts,NaN,NaN,NaN,NaN,NaN,NaN,[],2016.0,Bradley Botts,ZEDS Rising
8468,The Love Songs of W. E. B. Du Bois,Honorée Fanonne Jeffers,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,Honoree Fanonne Jeffers,The Love Songs of W E B Du Bois
10899,H.I.V.E.,Mark Walden,NaN,NaN,NaN,NaN,NaN,NaN,"['Young Adult', 'Science Fiction', 'Adventure'...",2006.0,Mark Walden,HIVE
15293,Seduce With Style 2.0,Vince Lin,NaN,NaN,NaN,NaN,NaN,NaN,[],2012.0,Vince Lin,Seduce With Style 20
21909,L.A. Weather,María Amparo Escandón,NaN,NaN,NaN,NaN,NaN,NaN,"['Fiction', 'Audiobook', 'Contemporary', 'Adul...",2021.0,Maria Amparo Escandon,LA Weather


In [59]:
missing_prosecraft.loc[:,'title_clean'] = missing_prosecraft['title'].str.replace('.', ' ')\
                                        .str.replace(' - ', ' ').str.replace('-',' ')\
                                        .str.replace('(',' ').str.replace(')',' ').str.replace(',',' ').str.strip()

In [60]:
missing_prosecraft.loc[:,'author_clean'] = missing_prosecraft['author'].str.replace('.',' ').str.replace('-',' ')

In [61]:
missing_prosecraft.sample(5)

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year,author_clean,title_clean
20926,Mars,Asja Bakić,NaN,NaN,NaN,NaN,NaN,NaN,"['Nonfiction', 'Self Help', 'Psychology', 'Rel...",1992.0,Asja Bakić,Mars
23777,Star Trek - Voyager: To Lose the Earth,Kirsten Beyer,NaN,NaN,NaN,NaN,NaN,NaN,"['Star Trek', 'Science Fiction', 'Star Trek Vo...",2019.0,Kirsten Beyer,Star Trek Voyager: To Lose the Earth
9222,In Veritas -,C. J. Lavigne,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,C J Lavigne,In Veritas
5545,Stories: All-New Tales,"Various Authors (ed, Neil Gaiman & Al Sarranto...",NaN,NaN,NaN,NaN,NaN,NaN,"['Short Stories', 'Fantasy', 'Fiction', 'Antho...",2010.0,"Various Authors (ed, Neil Gaiman & Al Sarranto...",Stories: All New Tales
4985,Star Trek - Discovery: Die Standing,John Jackson Miller,NaN,NaN,NaN,NaN,NaN,NaN,"['Star Trek', 'Science Fiction', 'Fiction', 'A...",2020.0,John Jackson Miller,Star Trek Discovery: Die Standing


This is progress, I think! Except now I've undone all my good work with the special characters I removed and replaced with letters before! Let's try looping through those again. 

In [62]:
missing_prosecraft.loc[23046]['title_clean']

'How To Have Kick Ass Ideas'

In [63]:
for before, after in replacements.items():
    missing_prosecraft.loc[:,'author_clean'] = missing_prosecraft['author_clean'].str.replace(before,after)
    missing_prosecraft.loc[:,'title_clean'] = missing_prosecraft['title_clean'].str.replace(before, after)

In [64]:
for before, after in (more_replacements).items():
    missing_prosecraft.loc[:,'title_clean'] = missing_prosecraft.title_clean.str.replace(before,after)
    missing_prosecraft.loc[:,'author_clean'] = missing_prosecraft.author_clean.str.replace(before,after)

In [65]:
missing_prosecraft.sample(5)

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year,author_clean,title_clean
5288,A New Idea of India,Harsh Madhusudan & Rajeev Mantri,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,Harsh Madhusudan and Rajeev Mantri,A New Idea of India
19828,Erotic Fantasy,Hans-Jürgen Döpp,NaN,NaN,NaN,NaN,NaN,NaN,"['Romance', 'Erotica', 'Fiction', 'Sexuality',...",2018.0,Hans Jurgen Dopp,Erotic Fantasy
23910,Heir of G.O.D.,Harper Maze,NaN,NaN,NaN,NaN,NaN,NaN,"['Dystopia', 'Science Fiction']",2020.0,Harper Maze,Heir of G O D
23046,How To Have Kick-Ass Ideas,Chris Baréz-Brown,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,Chris Barez Brown,How To Have Kick Ass Ideas
1729,Zipped,Laura McNeal & Tom McNeal,NaN,NaN,NaN,NaN,NaN,NaN,"['Young Adult', 'Fiction', 'Realistic Fiction'...",2003.0,Laura McNeal and Tom McNeal,Zipped


Noticed a weird one! It turns out there's two copies of a book on the actual Prosecraft website, and one of them is a formatting error. Instead of "Bombshell", by Stuart Woods & Parnell Hall, we have "& Parnell Hall - Bombshell" by Stuart Woods! The corrected version is already safely on the main dataframe.

In [66]:
missing_prosecraft.loc[18545]

title             & Parnell Hall - Bombshell
author                          Stuart Woods
total words                              NaN
vividness                                NaN
passive voice                            NaN
all adverbs                              NaN
ly-adverbs                               NaN
non-ly-adverbs                           NaN
genre                                     []
year                                     NaN
author_clean                    Stuart Woods
title_clean       and Parnell Hall Bombshell
Name: 18545, dtype: object

In [67]:
books[books['title'] == 'Bombshell']

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year
10270,Bombshell,Sarah Maclean,102513.0,40.84,8.3,3.64,1.80,1.84,"['Romance', 'Historical Romance', 'Historical'...",2021.0
11547,Bombshell,Stuart Woods & Parnell Hall,57004.0,34.21,10.2,2.41,0.72,1.69,"['Mystery', 'Fiction', 'Thriller', 'Crime', 'M...",2020.0


In [68]:
#Getting rid of the formatting error! 
missing_prosecraft = missing_prosecraft.drop(18545)

Time to take care of our special characters again! 

In [69]:
specials = set(missing_prosecraft.title_clean.sum()+missing_prosecraft.author_clean.sum()).difference(alphanum)
specials

{'!', '$', '(', ')', '+', ',', '-', '/', ':', '–', '’'}

In [70]:
for special in specials:
    missing_prosecraft.loc[:, 'title_clean'] = missing_prosecraft.title_clean.str.replace(special,'')
    missing_prosecraft.loc[:, 'author_clean'] = missing_prosecraft.author_clean.str.replace(special,'')

In [71]:
missing_prosecraft.sample(5)

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year,author_clean,title_clean
4830,Tough Luck L.A,Murray Sinclair,NaN,NaN,NaN,NaN,NaN,NaN,['Crime'],1981.0,Murray Sinclair,Tough Luck L A
21275,Star Wars - Aftermath: Life Debt,Chuck Wendig,NaN,NaN,NaN,NaN,NaN,NaN,"['Star Wars', 'Science Fiction', 'Fiction', 'F...",2016.0,Chuck Wendig,Star Wars Aftermath Life Debt
10899,H.I.V.E.,Mark Walden,NaN,NaN,NaN,NaN,NaN,NaN,"['Young Adult', 'Science Fiction', 'Adventure'...",2006.0,Mark Walden,H I V E
21909,L.A. Weather,María Amparo Escandón,NaN,NaN,NaN,NaN,NaN,NaN,"['Fiction', 'Audiobook', 'Contemporary', 'Adul...",2021.0,Maria Amparo Escandon,L A Weather
1491,Walk Through Walls,Marina Abramović,NaN,NaN,NaN,NaN,NaN,NaN,"['Art', 'Nonfiction', 'Memoir', 'Biography', '...",2016.0,Marina Abramovic,Walk Through Walls


Okay, we're ready to try running these through the function again! 

In [72]:
new_df = missing_prosecraft.progress_apply(lambda row: get_prosecraft(row.name), axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████| 84/84 [09:02<00:00,  6.45s/it]


In [73]:
len(new_df[new_df['vividness'].notna()])

53

Great! It looks like that's fixed 53 of the 85 missing values! 

In [74]:
in_order = new_df[['title', 'author', 'total words', 'vividness', 'passive voice',
       'all adverbs', 'ly-adverbs', 'non-ly-adverbs', 'genre', 'year']]

In [75]:
len(merged)

24776

In [76]:
len(pd.concat([merged,in_order]).dropna(subset=['vividness']))

24829

In [77]:
merged = pd.concat([merged,in_order]).dropna(subset=['vividness'])

In [78]:
#merged.to_csv('no_null_prosecraft.csv')

In [79]:
missing_prosecraft = missing_prosecraft[in_order['vividness'].isna()]

In [80]:
missing_prosecraft.head()

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year,author_clean,title_clean
892,ADHD 2.0,Edward M. Hallowell & John J. Ratey,NaN,NaN,NaN,NaN,NaN,NaN,"['Adhd', 'Nonfiction', 'Psychology', 'Self Hel...",2021.0,Edward M Hallowell and John J Ratey,ADHD 2 0
1491,Walk Through Walls,Marina Abramović,NaN,NaN,NaN,NaN,NaN,NaN,"['Art', 'Nonfiction', 'Memoir', 'Biography', '...",2016.0,Marina Abramovic,Walk Through Walls
1729,Zipped,Laura McNeal & Tom McNeal,NaN,NaN,NaN,NaN,NaN,NaN,"['Young Adult', 'Fiction', 'Realistic Fiction'...",2003.0,Laura McNeal and Tom McNeal,Zipped
2886,Morning noon and night,Sidney Sheldon,NaN,NaN,NaN,NaN,NaN,NaN,"['Fiction', 'Thriller', 'Mystery', 'Suspense',...",1995.0,Sidney Sheldon,Morning noon and night
3438,Star Wars - Galaxy’s Edge: Black Spire,Delilah S. Dawson,NaN,NaN,NaN,NaN,NaN,NaN,"['Star Wars', 'Science Fiction', 'Fiction', 'A...",2019.0,Delilah S Dawson,Star Wars Galaxys Edge Black Spire
3737,Creation Machine,Andrew Bannister,NaN,NaN,NaN,NaN,NaN,NaN,"['Science Fiction', 'Fiction', 'Space Opera', ...",2016.0,Andrew Bannister,Creation Machine
3977,Goosebumps - Slappy World: Please Do Not Feed ...,R. L. Stine,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,R L Stine,Goosebumps Slappy World Please Do Not Feed the...
4508,Magic Lies,C.C. Sommerly,NaN,NaN,NaN,NaN,NaN,NaN,[],2020.0,C C Sommerly,Magic Lies
5215,The Khruellian Encounter,C. J. Klinger,NaN,NaN,NaN,NaN,NaN,NaN,[],2014.0,C J Klinger,The Khruellian Encounter
7506,The Three Secret Cities,Matthew Reilly,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,Matthew Reilly,The Three Secret Cities


I'm later going to clear the ones where goodreads has no genre data, since genre is my target feature. Those will be the ones where the year exists but the genre list is empty. So let's not worry about fixing those ones! 

In [81]:
missing_prosecraft[(missing_prosecraft['genre'] == '[]') & (missing_prosecraft['year'].notna())]

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year,author_clean,title_clean
4508,Magic Lies,C.C. Sommerly,NaN,NaN,NaN,NaN,NaN,NaN,[],2020.0,C C Sommerly,Magic Lies
5215,The Khruellian Encounter,C. J. Klinger,NaN,NaN,NaN,NaN,NaN,NaN,[],2014.0,C J Klinger,The Khruellian Encounter
7806,Z.E.D.S. Rising,Bradley Botts,NaN,NaN,NaN,NaN,NaN,NaN,[],2016.0,Bradley Botts,Z E D S Rising


In [82]:
#Just grabbing the indices of the ones I don't need...
missing_prosecraft[(missing_prosecraft['genre'] == '[]') & (missing_prosecraft['year'].notna())].index

Index([4508, 5215, 7806], dtype='int64')

In [83]:
missing_prosecraft = missing_prosecraft.drop([4508, 5215, 7806])

In [84]:
missing_prosecraft.sample(5)

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year,author_clean,title_clean
7890,The Mind–Body Problem,Jonathan Westphal,NaN,NaN,NaN,NaN,NaN,NaN,"['Philosophy', 'Nonfiction', 'Science', 'Psych...",2016.0,Jonathan Westphal,The MindBody Problem
17079,The Girl Who Could Move Sh*t With Her Mind,Jackson Ford,NaN,NaN,NaN,NaN,NaN,NaN,"['Science Fiction', 'Fantasy', 'Mystery', 'Fic...",2019.0,Jackson Ford,The Girl Who Could Move Sht With Her Mind
23412,The Perfect Nine,Ngũgĩ wa Thiong’o,NaN,NaN,NaN,NaN,NaN,NaN,"['Fiction', 'Poetry', 'Africa', 'Mythology', '...",2018.0,Ngugi wa Thiongo,The Perfect Nine
22557,Minutes of Glory,Ngũgĩ wa Thiong’o,NaN,NaN,NaN,NaN,NaN,NaN,"['Short Stories', 'Fiction', 'Africa', 'Kenya'...",2019.0,Ngugi wa Thiongo,Minutes of Glory
8468,The Love Songs of W. E. B. Du Bois,Honorée Fanonne Jeffers,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,Honoree Fanonne Jeffers,The Love Songs of W E B Du Bois


Hmm... I've checked the URL of some of these and nothing seems to be unexpected! I'm not sure what's happening. Let's make a new function that just gives me the URLs.

In [85]:
def get_url(index):
    '''Given the cleaned title and author of a book, return the Prosecraft linguistic data'''
    
    #Copies the non-prosecraft columns of the original dataframe
    #row = missing_prosecraft[(missing_prosecraft['title_clean'] == title) & (missing_prosecraft['author_clean'] == author)]
    row = missing_prosecraft.loc[index]
    
    #Formats the URL
    title = row['title_clean'].replace(' ','-').lower()
    author = row['author_clean'].replace(' ','-').lower()
    URL = f"http://prosecraft.io/library/{author}/{title}"
        
    return URL

In [86]:
urls = missing_prosecraft.apply(lambda row: get_url(row.name), axis=1)

In [87]:
urls.iat[0]

'http://prosecraft.io/library/edward-m--hallowell-and-john-j--ratey/adhd-2-0'

Aha! It looks like several of the URLs have two spaces where there should only be one! I've also found a few with a trailing space. Let's see what we can do.

In [88]:
missing_prosecraft.loc[:,'author_clean'] = missing_prosecraft['author_clean'].str.replace('  ',' ').str.strip()
missing_prosecraft.loc[:,'title_clean'] = missing_prosecraft['title_clean'].str.replace('  ',' ').str.strip()

Let's try another round! 

In [89]:
new_df = missing_prosecraft.progress_apply(lambda row: get_prosecraft(row.name), axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [03:01<00:00,  6.48s/it]


In [90]:
in_order = new_df[['title', 'author', 'total words', 'vividness', 'passive voice',
       'all adverbs', 'ly-adverbs', 'non-ly-adverbs', 'genre', 'year']]

In [91]:
len(in_order[in_order['vividness'].notnull()])

19

In [92]:
merged = pd.concat([merged,in_order]).dropna(subset=['vividness'])
len(merged)

24848

In [93]:
#merged.to_csv('no_null_prosecraft.csv')

In [94]:
missing_prosecraft = missing_prosecraft[in_order['vividness'].isna()]

In [95]:
missing_prosecraft

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genre,year,author_clean,title_clean
1491,Walk Through Walls,Marina Abramović,NaN,NaN,NaN,NaN,NaN,NaN,"['Art', 'Nonfiction', 'Memoir', 'Biography', '...",2016.0,Marina Abramovic,Walk Through Walls
7890,The Mind–Body Problem,Jonathan Westphal,NaN,NaN,NaN,NaN,NaN,NaN,"['Philosophy', 'Nonfiction', 'Science', 'Psych...",2016.0,Jonathan Westphal,The MindBody Problem
14550,Belladonna,Daša Drndić,NaN,NaN,NaN,NaN,NaN,NaN,"['Fantasy', 'Romance', 'Young Adult', 'Mystery...",2022.0,Dasa Drndic,Belladonna
14708,Blinded by the Lights,Jakub Żulczyk,NaN,NaN,NaN,NaN,NaN,NaN,"['Polish Literature', 'Fiction', 'Thriller', '...",2014.0,Jakub Zulczyk,Blinded by the Lights
15171,Weep Not Child,Ngũgĩ wa Thiong’o,NaN,NaN,NaN,NaN,NaN,NaN,"['Africa', 'Fiction']",2005.0,Ngugi wa Thiongo,Weep Not Child
17079,The Girl Who Could Move Sh*t With Her Mind,Jackson Ford,NaN,NaN,NaN,NaN,NaN,NaN,"['Science Fiction', 'Fantasy', 'Mystery', 'Fic...",2019.0,Jackson Ford,The Girl Who Could Move Sht With Her Mind
20926,Mars,Asja Bakić,NaN,NaN,NaN,NaN,NaN,NaN,"['Nonfiction', 'Self Help', 'Psychology', 'Rel...",1992.0,Asja Bakic,Mars
22557,Minutes of Glory,Ngũgĩ wa Thiong’o,NaN,NaN,NaN,NaN,NaN,NaN,"['Short Stories', 'Fiction', 'Africa', 'Kenya'...",2019.0,Ngugi wa Thiongo,Minutes of Glory
23412,The Perfect Nine,Ngũgĩ wa Thiong’o,NaN,NaN,NaN,NaN,NaN,NaN,"['Fiction', 'Poetry', 'Africa', 'Mythology', '...",2018.0,Ngugi wa Thiongo,The Perfect Nine


Okay! Everyone left (except Jackson Ford and Jonathan Westphal, whose titles have mistreated special characters) has something in common--It turns out, with certain accented characters, prosecraft simply changes the letter into a space rather than the equivalent Latin character.

In [96]:
replacements['ũ'] = ' '
replacements['ĩ'] = ' '
replacements['Ż'] = ' '
replacements['ć'] = ' '

In [97]:
missing_prosecraft.loc[:,'title_clean'] = missing_prosecraft['title'].str.replace('.', ' ')\
                                        .str.replace(' - ', ' ').str.replace('-',' ')\
                                        .str.replace('(',' ').str.replace(')',' ').str.replace(',',' ')\
                                        .str.replace('*',' ').str.replace('–', ' ').str.replace('  ', ' ').str.strip()
missing_prosecraft.loc[:,'author_clean'] = missing_prosecraft['author'].str.replace('.', ' ')\
                                        .str.replace(' - ', ' ').str.replace('-',' ')\
                                        .str.replace('(',' ').str.replace(')',' ').str.replace(',',' ')\
                                        .str.replace('*',' ').str.replace('–', ' ').str.replace('  ', ' ').str.strip()

In [98]:
for before, after in replacements.items():
    missing_prosecraft.loc[:,'author_clean'] = missing_prosecraft['author_clean'].str.replace(before,after)
    missing_prosecraft.loc[:,'title_clean'] = missing_prosecraft['title_clean'].str.replace(before, after)
for before, after in more_replacements.items():
    missing_prosecraft.loc[:,'title_clean'] = missing_prosecraft.title_clean.str.replace(before,after)
    missing_prosecraft.loc[:,'author_clean'] = missing_prosecraft.author_clean.str.replace(before,after)

In [99]:
specials = set(missing_prosecraft.title_clean.sum()+missing_prosecraft.author_clean.sum()).difference(alphanum)
specials

{'’'}

In [100]:
#Just one special character left!
missing_prosecraft.loc[:,'author_clean'] = missing_prosecraft['author_clean'].str.replace('’','')

In [101]:
#Missed something the first time--some double spaces! 
missing_prosecraft.loc[:,'author_clean'] = missing_prosecraft['author_clean'].str.replace('  ',' ').str.strip()

Let's see if I got them all!

In [102]:
new_df = missing_prosecraft.progress_apply(lambda row: get_prosecraft(row.name), axis=1)
new_df

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [01:01<00:00,  6.88s/it]


,title,author,genre,year,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs
1491,Walk Through Walls,Marina Abramović,"['Art', 'Nonfiction', 'Memoir', 'Biography', '...",2016.0,114836.0,41.30,7.17,3.31,0.97,2.34
7890,The Mind–Body Problem,Jonathan Westphal,"['Philosophy', 'Nonfiction', 'Science', 'Psych...",2016.0,42756.0,27.19,8.97,3.46,1.37,2.09
14550,Belladonna,Daša Drndić,"['Fantasy', 'Romance', 'Young Adult', 'Mystery...",2022.0,113034.0,54.22,5.09,2.53,0.87,1.66
14708,Blinded by the Lights,Jakub Żulczyk,"['Polish Literature', 'Fiction', 'Thriller', '...",2014.0,154979.0,52.61,7.58,3.17,0.96,2.21
15171,Weep Not Child,Ngũgĩ wa Thiong’o,"['Africa', 'Fiction']",2005.0,40939.0,31.27,9.94,3.20,0.87,2.34
17079,The Girl Who Could Move Sh*t With Her Mind,Jackson Ford,"['Science Fiction', 'Fantasy', 'Mystery', 'Fic...",2019.0,120300.0,50.38,9.20,2.87,0.88,1.99
20926,Mars,Asja Bakić,"['Nonfiction', 'Self Help', 'Psychology', 'Rel...",1992.0,33749.0,43.13,9.43,3.69,1.53,2.15
22557,Minutes of Glory,Ngũgĩ wa Thiong’o,"['Short Stories', 'Fiction', 'Africa', 'Kenya'...",2019.0,59166.0,42.09,7.47,3.36,1.04,2.32
23412,The Perfect Nine,Ngũgĩ wa Thiong’o,"['Fiction', 'Poetry', 'Africa', 'Mythology', '...",2018.0,21807.0,61.12,5.61,2.35,0.56,1.79


SUCCESS! 

In [103]:
in_order = new_df[['title', 'author', 'total words', 'vividness', 'passive voice',
       'all adverbs', 'ly-adverbs', 'non-ly-adverbs', 'genre', 'year']]

In [104]:
merged = pd.concat([merged,in_order]).dropna(subset=['vividness'])

In [105]:
len(merged)

24857

In [106]:
merged.to_csv('no_null_prosecraft.csv')